In [1]:
import numpy as np
import torch
import pandas as pd
import pickle

from hypothesis_tests import chatterjee, benjamini_hochberg, NEWCORR, COR, DCOR, HSIC, HHG, TIC

In [2]:
test_name = HHG

In [3]:
ref_stats = torch.tensor(np.genfromtxt(f"results/spellman/{test_name}_reference_stats.txt"))
ref_pvals = torch.tensor(np.genfromtxt(f"results/spellman/{test_name}_reference_pvals.txt"))

# Subtract 1 moving from R indexing to Python indexing.
ref_idx = torch.sort(torch.tensor(np.genfromtxt(f"results/spellman/{test_name}_reference_inds.txt"), dtype=int) - 1)[0]

In [2]:
spellman = pd.read_csv("data/spellman_gene_expr_data.csv", header=0)
genes = list(spellman.columns[1:])

In [5]:
# stats = torch.zeros(len(genes))
# pvals = torch.zeros(len(genes))
# x = torch.tensor(spellman["time"].values)
# for i, gene in enumerate(genes):
#     y = torch.tensor(spellman[gene].values)
#     stats[i], pvals[i] = chatterjee(x, y, compute_pvalue=True)
# idx = torch.sort(benjamini_hochberg(pvals))[0]

pvals = torch.tensor(np.genfromtxt(f"results/spellman/{test_name}_pvalues.txt"))
idx = benjamini_hochberg(pvals)

In [29]:
def absolute_error(vals, ref_vals):
    return torch.max(torch.abs(vals - ref_vals))

def relative_error(vals, ref_vals):
    return torch.max(torch.abs((vals - ref_vals) / torch.median(ref_vals)))

In [7]:
# Difference in statistics.
# print("Absolute:", absolute_error(stats, ref_stats))
# print("Relative:", relative_error(stats, ref_stats))

In [8]:
# Difference in p-values.
print("Absolute:", absolute_error(pvals, ref_pvals))
print("Relative:", relative_error(pvals, ref_pvals))

Absolute: tensor(0.0050, dtype=torch.float64)
Relative: tensor(0.0446, dtype=torch.float64)


In [9]:
print("Absolute:", absolute_error(idx, ref_idx))

Absolute: tensor(0)


In [3]:
xionly = torch.sort(torch.tensor(np.genfromtxt(f"results/spellman/full_reference_xionly.txt"), dtype=int) - 1)[0]
notxi = torch.sort(torch.tensor(np.genfromtxt(f"results/spellman/full_reference_notxi.txt"), dtype=int) - 1)[0]

In [12]:
print(len(xionly))
print(len(notxi))

151
691


In [12]:
competitors = [
    COR,
    DCOR,
    # HSIC,
    HHG,
    TIC,
]


competitor_genes = set()
for method in competitors:
    pvals = torch.tensor(np.genfromtxt(f"results/spellman/{method}_reference_pvals.txt"))
    rejects_idx = benjamini_hochberg(pvals)
    new_genes = set([val.item() for val in rejects_idx])
    print("New length:\t", len(new_genes))
    competitor_genes = competitor_genes.union(new_genes)
    print("Total length:\t", len(competitor_genes))

print(len(competitor_genes))

New length:	 465
Total length:	 465
New length:	 494
Total length:	 550
New length:	 719
Total length:	 843
New length:	 291
Total length:	 853
853


In [16]:
rejects_idx = pickle.load(open("results/spellman/chatterjee_pt_genes.pkl", "rb"))
chat_genes = set([val.item() for val in rejects_idx])
len(chat_genes)

586

In [20]:
xionly_py = sorted(list(chat_genes - competitor_genes))
notxi_py =  sorted(list(competitor_genes - chat_genes))
print(len(xionly_py))
print(len(notxi_py))

248
515


In [24]:
xionly_py[0]

27

In [18]:
xionly = torch.sort(torch.tensor(np.genfromtxt(f"results/spellman/full_reference_xionly.txt"), dtype=int) - 1)[0]
notxi = torch.sort(torch.tensor(np.genfromtxt(f"results/spellman/full_reference_notxi.txt"), dtype=int) - 1)[0]

In [27]:
print(xionly[0:30])
print(xionly_py[0:30])

tensor([ 44,  96, 102, 139, 170, 171, 226, 229, 233, 234, 275, 317, 336, 362,
        371, 469, 478, 492, 499, 525, 596, 625, 675, 749, 831, 841, 918, 924,
        932, 955])
[27, 43, 95, 101, 138, 169, 170, 225, 226, 227, 228, 232, 233, 234, 264, 269, 274, 275, 290, 316, 334, 335, 361, 370, 401, 418, 419, 436, 448, 468]


In [30]:
for method in competitors:
    ref_pvals = torch.tensor(np.genfromtxt(f"results/spellman/{method}_reference_pvals.txt"))
    ref_idx = benjamini_hochberg(ref_pvals)
    pvals = torch.tensor(np.genfromtxt(f"results/spellman/{method}_pvalues.txt"))
    idx = benjamini_hochberg(pvals)
    print(absolute_error(idx, ref_idx))

tensor(0, dtype=torch.int32)
tensor(0, dtype=torch.int32)
tensor(0, dtype=torch.int32)
tensor(0, dtype=torch.int32)


In [31]:
len(xionly)

151

In [32]:
spellman.shape

(23, 4382)